In [1]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig

This can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64



## Meta Llama3 Base and Instruct

In [2]:
base_model_name = "meta-llama/Meta-Llama-3-8B"
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type='nf4',
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token
device_map = 'auto'
device = 'cuda'
model = AutoModelForCausalLM.from_pretrained(base_model_name, quantization_config=nf4_config,device_map=device_map)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
texts = [
    'kwsp itu apa?',
    "Apakah maksud perkataan 'integriti'?",
    "Terangkan perbezaan antara 'hormat' dan 'takut.'",
    "Translate the following English sentence to Malay: 'The cat is sitting on the table.'",
    "Translate the following Malay sentence to English: 'Saya suka makan nasi lemak.'",
    "Correct the following sentence: Me wants to go store.",
]

In [4]:
for text in texts:
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=64, pad_token_id=tokenizer.eos_token_id,do_sample=False)
    decoded = tokenizer.batch_decode(outputs)
    print('input:',text)
    print(decoded[0])
    print('***'*40)

/home/sani/miniconda3/envs/mergekit/lib/python3.10/site-packages/transformers/generation/utils.py:1510: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
2024-04-20 10:00:37.566776: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-20 10:00:38.277483: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enab

input: kwsp itu apa?
kwsp itu apa? (What is the name of the place?) (2) Apa nama kota itu? (What is the name of the city?) (3) Apa nama negara itu? (What is the name of the country?) (4) Apa nama pulau itu? (What is the name of the
************************************************************************************************************************


/home/sani/miniconda3/envs/mergekit/lib/python3.10/site-packages/transformers/generation/utils.py:1510: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


input: Apakah maksud perkataan 'integriti'?
Apakah maksud perkataan 'integriti'? (1 markah)
2. Apakah maksud perkataan 'kejujuran'? (1 markah)
3. Apakah maksud perkataan 'kebenaran'? (1 markah)
4. Apakah maksud perkataan 'keadilan'? (1 markah)
5
************************************************************************************************************************
input: Terangkan perbezaan antara 'hormat' dan 'takut.'
Terangkan perbezaan antara 'hormat' dan 'takut.'"
    "Explain the difference between 'hormat' and 'takut.'"
    "Menghormati" = "to respect"
    "Menghormati" = "to respect"
    "Menghormati" = "to respect"
    "Menghorm
************************************************************************************************************************
input: Translate the following English sentence to Malay: 'The cat is sitting on the table.'
Translate the following English sentence to Malay: 'The cat is sitting on the table.' (Use the appropriate pronoun.)
* [2] Translate the foll

## Basically Base ni bagus untuk generate next sentence bukan untuk menjawab soalan.

In [5]:
base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type='nf4',
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token
device_map = 'auto'
device = 'cuda'
model = AutoModelForCausalLM.from_pretrained(base_model_name, quantization_config=nf4_config,device_map=device_map)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
for text in texts:
    msg = [
        {"role": "user", "content": text},
    ]
    encodeds = tokenizer.apply_chat_template(msg, return_tensors="pt")
    outputs = model.generate(input_ids=encodeds.to("cuda"), max_new_tokens=64, pad_token_id=tokenizer.eos_token_id,do_sample=False)
    decoded = tokenizer.batch_decode(outputs)
    print('input:',text)
    print(decoded[0].split("<|end_header_id|>")[2].split("<|eot_id|>")[0])
    print('***'*40)

input: kwsp itu apa?


Kwsp is an abbreviation for "Kumpulan Wang Simpanan Pekerja" which is a Malay phrase that translates to "Employee Provident Fund" in English.

Kwsp is a mandatory savings plan in Malaysia that requires employers to contribute a certain percentage of their employees' salaries to
************************************************************************************************************************
input: Apakah maksud perkataan 'integriti'?


Perkataan "integriti" dalam bahasa Melayu bererti "kesucian" atau "kesederhanaan". Ia merujuk kepada sifat seseorang yang memiliki moral yang tinggi, jujur, dan tidak mudah terpengar
************************************************************************************************************************
input: Terangkan perbezaan antara 'hormat' dan 'takut.'


A great question!

In Malay, "hormat" and "takut" are two words that are often confused with each other, but they have distinct meanings.

**Hormat** (respect) refers to

## Kadang2 Instruct akan reply dalam bahasa melayu sebab dia memang dah tau conteks cuma dia diajar hanya untuk reply dalam English. Basically Base model dah tau tentang Malaysia(nasi lemak, kucing, etc) tapi time Instruct dia dah diajar untuk reply dalam English je.

## Masa untuk tunjuk beza finetune base dengan transformed instruct dari base untuk bahasa melayu

In [7]:
import gc
del model
gc.collect()
torch.cuda.empty_cache()

In [8]:
base_model_name = "../llm/model/"
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type='nf4',
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token
device_map = 'auto'
device = 'cuda'
model = AutoModelForCausalLM.from_pretrained(base_model_name, quantization_config=nf4_config,device_map=device_map)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
for text in texts:
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=64, pad_token_id=tokenizer.eos_token_id,do_sample=False)
    decoded = tokenizer.batch_decode(outputs)
    print('input:',text)
    print(decoded[0])
    print('***'*40)

input: kwsp itu apa?
kwsp itu apa? (What is the name of the place?) (2) Apa nama kota itu? (What is the name of the city?) (3) Apa nama negara itu? (What is the name of the country?) (4) Apa nama pulau itu? (What is the name of the
************************************************************************************************************************
input: Apakah maksud perkataan 'integriti'?
Apakah maksud perkataan 'integriti'? (1 markah)
2. Apakah maksud perkataan 'kejujuran'? (1 markah)
3. Apakah maksud perkataan 'kebenaran'? (1 markah)
4. Apakah maksud perkataan 'keadilan'? (1 markah)
5
************************************************************************************************************************
input: Terangkan perbezaan antara 'hormat' dan 'takut.'
Terangkan perbezaan antara 'hormat' dan 'takut.'"
    "Explain the difference between 'hormat' and 'takut.'"
    "Menghormati" = "to respect"
    "Menghormati" = "to respect"
    "Menghormati" = "to respect"
    "Menghorm
**

## Model dekat atas ni dah finetune llama3 base dengan bahasa melayu, tak ada banyak perbezaan dari segi output, tapi kalau inspect probability untuk next token akan bnyk berbeza

In [10]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "instruct_vector",device_map=device_map,quantization_config=nf4_config)

In [11]:
for text in texts:
    msg = [
        {"role": "user", "content": text},
    ]
    encodeds = tokenizer.apply_chat_template(msg, return_tensors="pt")
    outputs = model.generate(input_ids=encodeds.to("cuda"), max_new_tokens=64, pad_token_id=tokenizer.eos_token_id,do_sample=False)
    decoded = tokenizer.batch_decode(outputs)
    print('input:',text)
    print(decoded[0].split("<|end_header_id|>")[2].split("<|eot_id|>")[0])
    print('***'*40)

input: kwsp itu apa?


KWSP adalah singkatan dari "Kumpulan Wang Simpanan Pekerja" atau "Employee Provident Fund" dalam bahasa Inggris. KWSP adalah sebuah institusi yang bertugas mengumpulkan dan menyimpan dana pensiun untuk pekerja Malaysia. Dana tersebut akan digunakan untuk
************************************************************************************************************************
input: Apakah maksud perkataan 'integriti'?


Perkataan 'integriti' dalam bahasa Melayu bermaksud 'kesucian' atau 'kesahihan'. Ia juga dapat dikaitkan dengan 'kesetiaan' atau 'kesetiaan kepada nilai atau prinsip tertentu'.

Dalam kont
************************************************************************************************************************
input: Terangkan perbezaan antara 'hormat' dan 'takut.'


"Hormat" dan "takut" adalah dua kata yang memiliki makna yang berbeza, walaupun mereka memiliki beberapa kesamaan.

"Hormat" adalah kata yang bermakna "penghormatan", "penghargaan", atau 

## Tadaaa, Transformed llama3 Instruct dari base tadi dah bole jawab dalam Bahasa Melayu kalau tanya dalam BM. So far aku pakai model ni untuk experiment je, kalau ada berminat baru push ke huggingface